# Vector Field attempt
Plot horizontal velocity magnitude and angle
Almost got this working man

In [48]:
import xarray as xr
import hvplot.xarray
import numpy as np
import holoviews as hv
from holoviews import opts, dim
import datashader as ds
from JulesD3D.processNetCDF import fixMeshGrid, makeMeshGrid, makeVelocity, addDepth, addUnderlayerCoords, addVectorSum
from JulesD3D.utils import ncFilePicker, folderPicker
from JulesD3D.processing_2d import vector_angle
from os import path
from IPython.display import Markdown as md 
import ipywidgets as widgets
from glob import glob
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parent_folder = '/Users/julesblom/ThesisResults/'
# parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults'
scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom…

In [3]:
nc_filename = ncFilePicker(scenario_folder.value)
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [22]:
trim = xr.open_dataset(nc_filename.value)

In [23]:
trim = fixMeshGrid(trim, mystery_flag=True)
trim = addDepth(trim) # needs to be done BEFORE makeVelocity!
trim = makeVelocity(trim, transpose=False)

In [23]:
plot_settings = {
    'height': 600,
    'width': 444,
    'rasterize': True,
    'dynamic': True,
    'legend': True,
    'attr_labels': True,    
}

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


/Users/julesblom/Library/Caches/pypoetry/virtualenvs/delft3d-python-j5EFwY34-py3.7/lib/python3.7/site-packages/xarray/core/computation.py:1215: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return result.transpose(*[d for d in all_dims if d in result.dims])


## Add angle to dataset
* Similar issue: [MADIS netCDF to Pandas Dataframe: ValueError: iterator is too large](https://github.com/pydata/xarray/issues/838)
* Idea: make subset of trim with only necessary coords, dims and DataArrays

In [24]:
angle = vector_angle(trim.U1.values, trim.V1.values, trim.velocity.values)
angle.shape

/Users/julesblom/Library/Caches/pypoetry/virtualenvs/delft3d-python-j5EFwY34-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


(12, 80, 132, 182)

In [28]:
trim['angle'] = (('time', 'KMAXOUT_RESTR', 'M', 'N'), angle)
trim['angle'].attrs = {'long_name': 'Velocity angle per layer', 'units': 'Degrees', 'grid': 'grid', 'location': 'edge1'}
# trim['angle'] = trim.angle.assign_coords(depth_center=(('time', 'M', 'N', 'KMAXOUT_RESTR'), trim['depth_center'].values))

In [27]:
# blows up
# trim.isel(time=3, KMAXOUT_RESTR=-1).hvplot.vectorfield('XZ', 'YZ', angle='angle', mag='velocity', hover=False) #.opts(magnitude='velocity')

ValueError: iterator is too large

## HoloViews approach
Lacks the metadata and interactivity :'(

In [44]:
angle_values = trim.angle.isel(time=3, KMAXOUT_RESTR=-1).values
velocity_values = trim.velocity.isel(time=3, KMAXOUT_RESTR=-1).values

XZ = trim.XZ.values
YZ = trim.YZ.values

vector_data = (XZ, YZ, angle_values, velocity_values)

In [54]:
vectorfield = hv.VectorField(vector_data).opts(
    opts.Points(color='black', size=.1,), 
    opts.VectorField(title='Horizontal velocity angle', pivot='tip', magnitude='Magniture', color='Angle', height=600, width=444)
)
vectorfield

:VectorField   [x,y]   (Angle,Magnitude)

# Make clean subset DataSet of trim for hvPlot
* Copy M, N, XZ, YZ, time, KMAXOUT_RESTR to new DataSet
`hvplot.vectorfield` fails because 

In [59]:
trim.dims 

Frozen(SortedKeysDict({'MC': 132, 'NC': 182, 'M': 132, 'N': 182, 'LSTSCI': 2, 'LTUR': 2, 'SIG_LYR': 80, 'SIG_INTF': 81, 'KMAXOUT': 81, 'KMAXOUT_RESTR': 80, 'time': 12, 'LSEDTOT': 2, 'LSED': 2, 'nlyr': 77, 'nlyrp1': 78, 'avgtime': 1}))

In [60]:
trim.coords

Coordinates:
    XZ             (M, N) int64 0 0 0 0 0 0 ... 26100 26100 26100 26100 26100
    YZ             (M, N) int64 0 200 400 600 800 ... 35600 35800 36000 36200
  * SIG_LYR        (SIG_LYR) float32 -0.0359 -0.1018 ... -0.99955 -0.99985
  * SIG_INTF       (SIG_INTF) float32 0.0 -0.0718 -0.1318 ... -0.9997 -1.0
  * KMAXOUT        (KMAXOUT) int32 0 1 2 3 4 5 6 7 8 ... 73 74 75 76 77 78 79 80
  * KMAXOUT_RESTR  (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 73 74 75 76 77 78 79
  * time           (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
    depth          (time, M, N, SIG_INTF) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    depth_center   (time, M, N, SIG_LYR) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [65]:
ds = xr.Dataset()

ds.coords['KMAXOUT_RESTR'] = trim.KMAXOUT_RESTR
ds.coords['time'] = trim.time
ds.coords['XZ'] = trim.XZ
ds.coords['YZ'] = trim.YZ
ds['angle'] = (('time', 'KMAXOUT_RESTR', 'M', 'N'), angle)
ds['angle'].attrs = {'long_name': 'Velocity angle per layer', 'units': 'Degrees', 'grid': 'grid', 'location': 'edge1'}
ds['velocity'] = trim.velocity

ds

<xarray.Dataset>
Dimensions:        (KMAXOUT_RESTR: 80, M: 132, N: 182, time: 12)
Coordinates:
  * KMAXOUT_RESTR  (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 73 74 75 76 77 78 79
  * time           (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
    YZ             (M, N) int64 0 200 400 600 800 ... 35600 35800 36000 36200
    XZ             (M, N) int64 0 0 0 0 0 0 ... 26100 26100 26100 26100 26100
Dimensions without coordinates: M, N
Data variables:
    angle          (time, KMAXOUT_RESTR, M, N) float32 nan nan nan ... nan nan
    velocity       (time, KMAXOUT_RESTR, M, N) float32 0.0 0.0 0.0 ... 0.0 0.0

In [ ]:
ds.isel(KMAXOUT_RESTR=-1).hvplot.vectorfield('XZ', 'YZ', angle='angle', mag='velocity', hover=False, **plotting_options) #.opts(magnitude='velocity')